## Use Cash-Karp Runge-Kutta method with adaptive stepwise control to repeat exercise from 11/6

## Evolve the system of equations dy/dx = z   dz/dx =-y use the intial conditons y(x=0) = 0 and dydx(x=0) = 1 and evolve over the range [0,2*pi].

## Plot the solutions y(x) and dy/dx(x) over the range and the numerical solution

## Plot the absolute error for the numerical solutions y(x) dy/dx(x) over the range



In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def dydx(x,y):
    
    y_derivs = np.zeros(2)
    
    y_derivs[0] = y[1]
    
    y_derivs[1] = -1*y[0]
    
    return y_derivs

In [ ]:
def rk5_mv_core(dydx,xi,yi,nv,h):
    
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    x_ipoh = xi + 0.5*h
    
    x_ipo = xi + h
    
    y_temp = np.zeros(nv)
    
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    y_temp[:] = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    
    y_temp[:] = yi[:] + 0.5*k2[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k3[:] = h*y_derivs[:]
    
    y_temp[:] = yi[:] + 0.5*k3[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k4[:] = h*y_derivs[:]
    
    y_temp[:] = yi[:] + 0.5*k4[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k5[:] = h*y_derivs[:]
    
    y_temp[:] = yi[:] + k5[:]
    y_derivs = dydx(x_ipo,y_temp)
    k6[:] = h*y_derivs
    
    yipo = yi + (16*k1/135 + 1408*k3/2565 + 2197*k4/4104 - 1*k5/5)
    yipoa = yi + (16*k1/135 + 6656*k3/12825 + 28561*k4/56340 - 9*k5/50 + 2*k6/55)
    
    return yipo
    return yipoa
   

In [ ]:
def rk5_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    SAFETY    = 0.8
    H_NEW_FAC = 2.0
    
    imax = 10000
    
    i = 0 
    
    Epsilon = np.full(nv,2*tol)
    
    h_step = h
    
    while(Epsilon.max()/tol > 1.0):
        
        y_2  = rk5_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1  = rk5_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk5_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step)
        
        Epsilon = np.fabs(y_2 - y_11)
        
        if(Epsilon.max()/tol > 1.0):
            
            h_step *= SAFETY * (Epsilon.max()/tol)**(-0.25)
            
        if(i>=imax):
            print("Too many iterations in rk5_mv_ad()")
            raise StopIterations("Ending after i = ",i)
            
        i+=1 
        
    h_new = np.fmin(h_step * (Epsilon.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    return y_2, h_new, h_step

In [ ]:
def rk5_mv(dfdx,a,b,y_a,tol):
    
    xi = a
    yi = y_a.copy()
    
    h = 1.0e-4 * (b-a)
    
    imax = 10000
    
    i = 0
    
    nv = len(y_a)
    
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    flag = 1
    
    while(flag):
        
        yi_new, h_new, h_step = rk5_mv_ad(dydx,xi,yi,nv,h,tol)
        
        h = h_new
        
        if(xi+h_step>b):
            
            h= b-xi
            
            yi_new, h_new, h_step = rk5_mv_ad(dydx,xi,yi,nv,h,tol)
            
            flag = 0
        
        xi += h_step
        yi[:] = yi_new[:]
        
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1:] = yi[:]
        del y
        y = y_new
        
        if(i>=imax):
            
            print("Maximum iterations reached")
            raise StopIteration("Iteration number = ",i)
            
        i += 1
        
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi, h_step, b)
        print(s)
        
        if(xi==b):
            flag = 0
            
    return x,y

In [ ]:
a = 0.0
b = 2.0 * np.pi
y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2 

tolerance = 1.0e-6

x,y = rk5_mv(dydx,a,b,y_0,tolerance)

In [ ]:
plt.plot(x,y[:,0],'yo',label='y(x)')
plt.plot(x,y[:,1],'bo',label='dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx,np.sin(xx),label='sin(x)')
plt.plot(xx,np.cos(xx),label='cos(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0]-sine)
dydx_error = (y[:,1]-cosine)

plt.plot(x, y_error,'y', label="y(x) Error")
plt.plot(x, dydx_error,'b', label="dydx(x) Error")
plt.legend(frameon=False)